In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import os
import cv2
import joblib
from tqdm.notebook import tqdm

In [22]:
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, auc, accuracy_score, roc_curve, RocCurveDisplay, confusion_matrix
from sklearn.preprocessing import LabelEncoder, LabelBinarizer,Label_binarize
from sklearn.model_selection import GridSearchCv, train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.model import Model, Sequential, save_model
from tensorflow.keras.layer import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling

In [15]:
train_labels = pd.read_csv("/kaggle/input/raf-db-dataset/train_labels.csv")
test_labels = pd.read_csv("/kaggle/input/raf-db-dataset/test_labels.csv")

In [16]:
train_labels.head()

,image,label
0,train_00001_aligned.jpg,5
1,train_00002_aligned.jpg,5
2,train_00003_aligned.jpg,4
3,train_00004_aligned.jpg,4
4,train_00005_aligned.jpg,5


In [17]:
test_labels.head()

,image,label
0,test_0001_aligned.jpg,5
1,test_0002_aligned.jpg,1
2,test_0003_aligned.jpg,4
3,test_0004_aligned.jpg,1
4,test_0005_aligned.jpg,5


In [11]:
classes = ["surprise","fear","disgust","happy","sad","angry","neutral"]
label_map = {label:(idx+1) for idx,label in enumerate(classes)}
label_map

{'surprise': 1,
 'fear': 2,
 'disgust': 3,
 'happy': 4,
 'sad': 5,
 'angry': 6,
 'neutral': 7}